<img style="float: left;;" src='Figures/iteso.jpg' width="100" height="200"/></a>

# <center> <font color= #000047> Transformaciones Lineales </font> </center>

In [ ]:
import pandas as pd

from sklearn.datasets import fetch_california_housing
ch=fetch_california_housing()
df=pd.DataFrame(ch.data,columns=ch.feature_names)
df['MedHouseVal']=ch.target
df.head(2)

# Estandarización

Desplaza la media $\mu$ de $X$ a cero y escala la varianza $\sigma$ a uno
$$
X'=\frac{X-\mu}{\sigma},
$$
donde $X'$ es la variable transformada con media cero y varianza uno.

In [ ]:
import numpy as np
from scipy.stats import zscore
from sklearn.preprocessing import StandardScaler

In [ ]:
df.describe()

In [ ]:
df_est=pd.DataFrame(zscore(df))
df_est.describe()

In [ ]:
me=np.mean(df,axis=0)
ds=np.std(df)

In [ ]:
me

In [ ]:
ds

In [ ]:
esc_est=StandardScaler()
pd.DataFrame(esc_est.fit_transform(df),columns=df.columns).describe()

In [ ]:
esc_est.mean_ # Medias originales

In [ ]:
esc_est.scale_ # Desviaciones estandar originales

# Escalamiento con base en la media

Desplaza la media $\mu$ de $X$ a cero y el rango a uno
$$
X'=\frac{X-\mu}{max(X)-min(X)}.
$$

In [ ]:
rng=df.max()-df.min()
df_med=(df-me)/rng

In [ ]:
rng

In [ ]:
df_med.describe()

# Escalamiento Robusto (Por cuantiles)
Desplaza la mediana $\bar{x}$ de $X$ a cero y el rango intercuartílico a uno
$$
X'=\frac{X-\bar{x}}{Q_3-Q_1},
$$
donde $Q_1$ y $Q_3$ son los cuartiles uno y tres respectivamente.

In [ ]:
from sklearn.preprocessing import RobustScaler

In [ ]:
mediana=df.median()
mediana

In [ ]:
Q=df.quantile((0.25,0.75))
iqr=Q.loc[0.75]-Q.loc[0.25]
iqr

In [ ]:
df_rob=(df-mediana)/iqr
df_rob.describe()

In [ ]:
Q_rob=df_rob.quantile((0.25,0.75))
Q_rob.loc[0.75]-Q_rob.loc[0.25]

In [ ]:
esc_rob=RobustScaler()
pd.DataFrame(esc_rob.fit_transform(df),columns=df.columns).describe()

In [ ]:
esc_rob.center_ # Mediana de las variables originales

In [ ]:
esc_rob.scale_ # IQR de las variables originales

# Normalización de mínimo y máximo
Queremos que nuestros datos esten entre (0,1).
$$
X'=\frac{X-\min{X}}{\max{X}-\min{X}}.
$$

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
m=df.min()
M=df.max()
df_minMax=(df-m)/(M-m)
df_minMax.describe()

In [ ]:
A,B=5,10
df_AB=A+(B-A)*(df-m)/(M-m)
df_AB.describe()

In [ ]:
esc_minMax=MinMaxScaler((A,B))
pd.DataFrame(esc_minMax.fit_transform(df),columns=df.columns).describe()

In [ ]:
esc_minMax.min_ # Mínimo de las variables originales

In [ ]:
esc_minMax.scale_ # Rango de las variables originales

# Escalamiento de máximo absoluto
$$
X'=\frac{X}{\max{|X|}}.
$$

Se recomienda para datos centrados en cero o para variables con pocas observaciones.

In [ ]:
from sklearn.preprocessing import MaxAbsScaler

In [ ]:
df.describe()

In [ ]:
df_max=df/(np.max(np.abs(df),axis=0))
df_max.describe()

In [ ]:
# Escalamiento sobre la variable centrada
#
df_centrada=df-me
df_maxCentrada=df_centrada/np.max(np.abs(df_centrada),axis=0)
df_maxCentrada.describe()

In [ ]:
esc_maxAbs=MaxAbsScaler()
pd.DataFrame(esc_maxAbs.fit_transform(df),columns=df.columns).describe()

In [ ]:
pd.DataFrame(esc_maxAbs.fit_transform(df_centrada),columns=df.columns).describe()

In [ ]:
esc_maxAbs.max_abs_

In [ ]:
esc_maxAbs.scale_

# Regresión

In [ ]:
from scipy.stats import yeojohnson
from sklearn.linear_model import LinearRegression

In [ ]:
# Antes de hacer la separación de datos entre train y test, necesitamos hacer la transformación de potencia
#
df_t=df.copy()
lmb={}
for var in df:
  x=(df[var]-df[var].mean())/(df[var].max()-df[var].min())
  df_t[var],lmb[var]=yeojohnson(x)
H=df_t.hist(bins='auto',figsize=(10,8))

In [ ]:
df_t.boxplot(figsize=(10,7))

In [ ]:
df_tEsc=df.copy()
vars_rob=['MedInc','AveRooms','AveBedrms','AveOccup']
df_tEsc[vars_rob]=esc_rob.fit_transform(df[vars_rob])
df_tEsc.describe()

In [ ]:
vars_est=['HouseAge','Population','Latitude','Longitude','MedHouseVal']
df_tEsc[vars_est]=esc_est.fit_transform(df[vars_est])
df_tEsc.describe()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X,x,Y,y=train_test_split(df_tEsc.drop(columns='MedHouseVal'),df_tEsc['MedHouseVal'])
lin=LinearRegression()
lin.fit(X,Y)

In [ ]:
lin.score(X,Y),lin.score(x,y)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Graficamos 'MedInc' (0), 'Population' (4) y 'Longitude' (7)
#
x=df_tEsc['MedInc']
plt.scatter(x,df_tEsc['MedHouseVal'],s=5)
plt.plot(x,lin.intercept_+lin.coef_[0]*x,'r')
plt.grid()

In [ ]:
x=df_tEsc['Population']
plt.scatter(x,df_tEsc['MedHouseVal'],s=5)
plt.plot(x,lin.intercept_+lin.coef_[4]*x,'r')
plt.grid()

In [ ]:
x=df_tEsc['Longitude']
plt.scatter(x,df_tEsc['MedHouseVal'],s=5)
plt.plot(x,lin.intercept_+lin.coef_[7]*x,'r')
plt.grid()

In [ ]:
# Regresión con los datos sin transformar
X_ST,x,Y_ST,y=train_test_split(df.drop(columns='MedHouseVal'),df['MedHouseVal'])
lin_ST=LinearRegression()
lin_ST.fit(X_ST,Y_ST)
lin_ST.score(X_ST,Y_ST),lin_ST.score(x,y)

In [ ]:
x=df['MedInc']
plt.scatter(x,df['MedHouseVal'],s=5)
plt.plot(x,lin_ST.intercept_+lin_ST.coef_[0]*x,'r')
plt.grid()

In [ ]:
x=df['Population']
plt.scatter(x,df['MedHouseVal'],s=5)
plt.plot(x,lin_ST.intercept_+lin_ST.coef_[4]*x,'r')
plt.grid()

In [ ]:
x=df['Longitude']
plt.scatter(x,df['MedHouseVal'],s=5)
plt.plot(x,lin_ST.intercept_+lin_ST.coef_[7]*x,'r')
plt.grid()

In [ ]:
df.describe()